<a href="https://colab.research.google.com/github/DongheeKang/MachineLearning/blob/master/PyTorch_Vision_transformer_DeiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing Vision Transformer Model

What is DeiT? 

CNNs typically require hundreds of millions of images for training to achieve the state-of-the-art SOTA results. DeiT is a vision transformer model that requires a lot less
data and computing resources for training to compete with the leading
CNNs in performing image classification, which is made possible by two
key components of of DeiT:

-  Data augmentation that simulates training on a much larger dataset;
-  Native distillation that allows the transformer network to learn from
   a CNN’s output.

DeiT shows that Transformers can be successfully applied to computer
vision tasks, with limited access to data and resources.

Facebook Data-efficient Image Transformers `DeiT <https://ai.facebook.com/blog/data-efficient-image-transformers-a-promising-new-technique-for-image-classification>`:


How to use the DeiT? 

Vision Transformer models apply the cutting-edge attention-based
transformer models, introduced in Natural Language Processing to achieve
all kinds of the SOTA results, to Computer Vision tasks. 

* compare the performance of quantized, optimized and non-quantized, non-optimized
models, 
* show the benefits of applying quantization and optimization
to the model along the steps.


Classifying Images with DeiT
-------------------------------

Follow the README at the DeiT repo for detailed information on how to
classify images using DeiT, or for a quick test, first install the
required packages:



In [1]:
# pip install torch torchvision timm pandas requests

To run in Google Colab, uncomment the following line:



In [7]:
!pip install timm pandas requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 14.8 MB/s 


then run the script below:



In [8]:
from PIL import Image
import torch
import timm

import requests
import torchvision.transforms as transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

%matplotlib inline

print(torch.__version__)
# should be 1.8.0


model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

img = Image.open(requests.get("https://raw.githubusercontent.com/pytorch/ios-demo-app/master/HelloWorld/HelloWorld/HelloWorld/image.png", stream=True).raw)
img = transform(img)[None,]
out = model(img)
clsidx = torch.argmax(out)
print(clsidx.item())

1.12.0+cu113


Downloading: "https://github.com/facebookresearch/deit/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth


  0%|          | 0.00/330M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


269


The output should be 269, which, according to the ImageNet list of class
index to `labels file <https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a>`_, maps to ‘timber
wolf, grey wolf, gray wolf, Canis lupus’.

Now that we have verified that we can use the DeiT model to classify
images, let’s see how to modify the model so it can run on iOS and
Android apps.




Scripting DeiT
----------------------
To use the model on mobile, we first need to script the
model. See the `Script and Optimize recipe <https://pytorch.org/tutorials/recipes/script_optimized.html>`_ for a
quick overview. Run the code below to convert the DeiT model used in the
previous step to the TorchScript format that can run on mobile.




In [9]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()
scripted_model = torch.jit.script(model)
scripted_model.save("fbdeit_scripted.pt")

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


The scripted model file fbdeit_scripted.pt of size about 346MB is
generated.




Quantizing DeiT
---------------------
To reduce the trained model size significantly while
keeping the inference accuracy about the same, quantization can be
applied to the model. Thanks to the transformer model used in DeiT, we
can easily apply dynamic-quantization to the model, because dynamic
quantization works best for LSTM and transformer models (see `here <https://pytorch.org/docs/stable/quantization.html?highlight=quantization#dynamic-quantization>`_
for more details).

Now run the code below:




In [10]:
# Use 'fbgemm' for server inference and 'qnnpack' for mobile inference
backend = "fbgemm" # replaced with qnnpack causing much worse inference speed for quantized model on this notebook
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

quantized_model = torch.quantization.quantize_dynamic(model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_quantized_model = torch.jit.script(quantized_model)
scripted_quantized_model.save("fbdeit_scripted_quantized.pt")

/usr/local/lib/python3.7/dist-packages/torch/ao/quantization/observer.py:178: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


This generates the scripted and quantized version of the model
fbdeit_quantized_scripted.pt, with size about 89MB, a 74% reduction of
the non-quantized model size of 346MB!




You can use the ``scripted_quantized_model`` to generate the same
inference result:




In [11]:
out = scripted_quantized_model(img)
clsidx = torch.argmax(out)
print(clsidx.item())
# The same output 269 should be printed

269


Optimizing DeiT
---------------------
The final step before using the quantized and scripted
model on mobile is to optimize it:




In [12]:
from torch.utils.mobile_optimizer import optimize_for_mobile
optimized_scripted_quantized_model = optimize_for_mobile(scripted_quantized_model)
optimized_scripted_quantized_model.save("fbdeit_optimized_scripted_quantized.pt")

The generated fbdeit_optimized_scripted_quantized.pt file has about the
same size as the quantized, scripted, but non-optimized model. The
inference result remains the same.




In [13]:
out = optimized_scripted_quantized_model(img)
clsidx = torch.argmax(out)
print(clsidx.item())
# Again, the same output 269 should be printed

269


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1405.)
  return forward_call(*input, **kwargs)


Using Lite Interpreter
------------------------

To see how much model size reduction and inference speed up the Lite
Interpreter can result in, let’s create the lite version of the model.




In [14]:
optimized_scripted_quantized_model._save_for_lite_interpreter("fbdeit_optimized_scripted_quantized_lite.ptl")
ptl = torch.jit.load("fbdeit_optimized_scripted_quantized_lite.ptl")

Although the lite model size is comparable to the non-lite version, when
running the lite version on mobile, the inference speed up is expected.




Comparing Inference Speed
---------------------------

To see how the inference speed differs for the four models - the
original model, the scripted model, the quantized-and-scripted model,
the optimized-quantized-and-scripted model - run the code below:




In [15]:
with torch.autograd.profiler.profile(use_cuda=False) as prof1:
    out = model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof2:
    out = scripted_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof3:
    out = scripted_quantized_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof4:
    out = optimized_scripted_quantized_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof5:
    out = ptl(img)

print("original model: {:.2f}ms".format(prof1.self_cpu_time_total/1000))
print("scripted model: {:.2f}ms".format(prof2.self_cpu_time_total/1000))
print("scripted & quantized model: {:.2f}ms".format(prof3.self_cpu_time_total/1000))
print("scripted & quantized & optimized model: {:.2f}ms".format(prof4.self_cpu_time_total/1000))
print("lite model: {:.2f}ms".format(prof5.self_cpu_time_total/1000))

original model: 1837.46ms
scripted model: 1688.36ms
scripted & quantized model: 555.15ms
scripted & quantized & optimized model: 514.35ms
lite model: 517.10ms


The following results summarize the inference time taken by each model
and the percentage reduction of each model relative to the original
model.




In [16]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'Model': ['original model','scripted model', 'scripted & quantized model', 'scripted & quantized & optimized model', 'lite model']})
df = pd.concat([df, pd.DataFrame([
    ["{:.2f}ms".format(prof1.self_cpu_time_total/1000), "0%"],
    ["{:.2f}ms".format(prof2.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof2.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof3.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof3.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof4.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof4.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof5.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof5.self_cpu_time_total)/prof1.self_cpu_time_total*100)]],
    columns=['Inference Time', 'Reduction'])], axis=1)

print(df)

"""
        Model                             Inference Time    Reduction
0	original model                             1236.69ms           0%
1	scripted model                             1226.72ms        0.81%
2	scripted & quantized model                  593.19ms       52.03%
3	scripted & quantized & optimized model      598.01ms       51.64%
4	lite model                                  600.72ms       51.43%
"""

                                    Model Inference Time Reduction
0                          original model      1837.46ms        0%
1                          scripted model      1688.36ms     8.11%
2              scripted & quantized model       555.15ms    69.79%
3  scripted & quantized & optimized model       514.35ms    72.01%
4                              lite model       517.10ms    71.86%


'\n        Model                             Inference Time    Reduction\n0\toriginal model                             1236.69ms           0%\n1\tscripted model                             1226.72ms        0.81%\n2\tscripted & quantized model                  593.19ms       52.03%\n3\tscripted & quantized & optimized model      598.01ms       51.64%\n4\tlite model                                  600.72ms       51.43%\n'

Learn More
~~~~~~~~~~~~~~~~~

- `Facebook Data-efficient Image Transformers <https://ai.facebook.com/blog/data-efficient-image-transformers-a-promising-new-technique-for-image-classification>`__
- `Vision Transformer with ImageNet and MNIST on iOS <https://github.com/pytorch/ios-demo-app/tree/master/ViT4MNIST>`__
- `Vision Transformer with ImageNet and MNIST on Android <https://github.com/pytorch/android-demo-app/tree/master/ViT4MNIST>`__

